# 前処理大全 SQL+Python版 (Chapter7)

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# MagicコマンドでSQLを書くための設定
%sql $dsl

'Connected: padawan@dsdojo_db'

In [2]:
import pandas as pd
from scipy.sparse import csc_matrix
customer_df = pd.read_csv("./data/customer.csv")
reserve_df = pd.read_csv("./data/reserve.csv")

# 演習問題

### 007_spread/001: 集計結果のマトリクス化

#### SQL (Not awesome)
- レコードを顧客IDごとの宿泊人数ごとの予約数内訳となるような集計表を作成せよ。

In [3]:
%%sql
WITH count_table AS (
    SELECT
        customer_id
        ,people_num
        ,COUNT(reserve_id) AS reserve_count
    FROM reserve_tb
    GROUP BY customer_id, people_num
)
SELECT
    customer_id
    ,MAX(CASE people_num WHEN 1 THEN reserve_count ELSE 0 END) AS people_num_1
    ,MAX(CASE people_num WHEN 2 THEN reserve_count ELSE 0 END) AS people_num_2
    ,MAX(CASE people_num WHEN 3 THEN reserve_count ELSE 0 END) AS people_num_3
    ,MAX(CASE people_num WHEN 4 THEN reserve_count ELSE 0 END) AS people_num_4
FROM count_table
GROUP BY customer_id
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,people_num_1,people_num_2,people_num_3,people_num_4
c_96,0,1,2,0
c_353,1,2,0,0
c_897,0,0,0,1
c_909,1,4,1,2
c_581,2,1,0,0
c_140,1,2,2,2
c_266,2,2,2,2
c_271,1,3,2,1
c_980,0,0,3,3
c_698,1,0,2,1


- 最初のサブクエリで2つのGROUP BYキーを指定する。
- カラム名にする方のキー毎にCASE文を使ってうまくデータ化する。
- not awesomeだけど実用では使う場合あるかもしれぬ。

#### Python (Awesome)

In [24]:
df = pd.pivot_table(
    reserve_df, index='customer_id', columns='people_num', values='reserve_id',
    aggfunc=lambda x: len(x),
    fill_value=0,
)
df

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1
...,...,...,...,...
c_994,1,0,0,0
c_995,2,2,1,3
c_996,0,4,3,0


- customer_idがindexとなり、~columnもmulti columnsになってしまうのが気に入らねぇ。ので。~
- multi columnsではないみたい。カラム自体に名前が付与されている。

In [5]:
df.columns = [1,2,3,4]
df = df.reset_index(drop=False)
df

,customer_id,1,2,3,4
0,c_1,2,2,2,2
1,c_10,0,2,2,2
2,c_100,2,1,2,0
3,c_1000,1,0,0,1
4,c_101,2,1,1,1
...,...,...,...,...,...
883,c_994,1,0,0,0
884,c_995,2,2,1,3
885,c_996,0,4,3,0
886,c_997,0,1,1,0


### 007_spread/002: スパース場合のマトリクス化

#### Python (Awesome)

In [11]:
count_table = reserve_df \
    .groupby(['customer_id', 'people_num'], as_index=False).reserve_id.size()
count_table.columns = ['customer_id', 'people_num', 'reserve_count']
count_table

,customer_id,people_num,reserve_count
0,c_1,1,2
1,c_1,2,2
2,c_1,3,2
3,c_1,4,2
4,c_10,2,2
...,...,...,...
2361,c_997,3,1
2362,c_999,1,1
2363,c_999,2,3
2364,c_999,3,1


- pandasでもCategoricalEncodeができるのかー。

In [39]:
coustomer_id_categories = pd.Categorical(count_table['customer_id'])
print(coustomer_id_categories.codes)

people_num_categories = pd.Categorical(count_table['people_num'])
print(people_num_categories.codes)

[  0   0   0 ... 887 887 887]
[0 1 2 ... 1 2 3]


- スパース型は平たく言うと、ゼロじゃないところの値とゼロじゃないところの縦横のindex番号で表現した行列。
- 0が多いデータの場合、この方がメモリを消費せずにデータを保持できる。
- 自然言語処理で頻出する。なぜならば単語の種類数とかがカラムになる場合があるから。

In [41]:
csc_matrix(
    (count_table['reserve_count'],
         (coustomer_id_categories.codes,
          people_num_categories  .codes))
    ,shape=(
        len(coustomer_id_categories), len(people_num_categories)
    )
)

<2366x2366 sparse matrix of type '<class 'numpy.int64'>'
	with 2366 stored elements in Compressed Sparse Column format>

### ワイがすきなのは、sklearn風に使えるcategory encoders

In [6]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 5.0 MB/s eta 0:00:011


In [7]:
import category_encoders as ce

In [15]:
count_table['customer_id'].values

array(['c_1', 'c_1', 'c_1', ..., 'c_999', 'c_999', 'c_999'], dtype=object)

In [23]:
oe = ce.OrdinalEncoder(cols=['customer_id', 'people_num'], handle_unknown='ignore')
oe.fit_transform(count_table)

,customer_id,people_num,reserve_count
0,1,1,2
1,1,2,2
2,1,3,2
3,1,4,2
4,2,2,2
...,...,...,...
2361,887,3,1
2362,888,1,1
2363,888,2,3
2364,888,3,1
